In [1]:
import os
import random
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from glob import glob
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import xgboost as xgb
from torchvision import models, transforms
from PIL import Image
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm.notebook import tqdm
import numpy as np

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# 재현성을 위해 시드 설정
SEED = 85
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

In [3]:
# 데이터 불러오기
labels_train_val = pd.read_csv('../../../../..//data/train_val_list.txt')
labels_train_val.columns = ['Image_Index']
labels_test = pd.read_csv('../../../../..//data/test_list.txt')
labels_test.columns = ['Image_Index']

disease_labels = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening',
'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'No Finding']

labels_df = pd.read_csv('../../../../..//data/Data_Entry_2017.csv')
labels_df.columns = ['Image_Index', 'Finding_Labels', 'Follow_Up_#', 'Patient_ID',
                     'Patient_Age', 'Patient_Gender', 'View_Position',
                     'Original_Image_Width', 'Original_Image_Height',
                     'Original_Image_Pixel_Spacing_X',
                     'Original_Image_Pixel_Spacing_Y', 'dfd']

# One hot encoding
for disease in tqdm(disease_labels):
    labels_df[disease] = labels_df['Finding_Labels'].map(lambda result: 1 if disease in result else 0)

labels_df['Finding_Labels'] = labels_df['Finding_Labels'].apply(lambda s: [l for l in str(s).split('|')])

num_glob = glob('../../../../../data/images_all/*.png')
img_path = {os.path.basename(x): x for x in num_glob}

labels_df['Paths'] = labels_df['Image_Index'].map(img_path.get)

train_val_df = labels_df[labels_df['Image_Index'].isin(labels_train_val['Image_Index'])]
test_df = labels_df[labels_df['Image_Index'].isin(labels_test['Image_Index'])]

print('train_val size', train_val_df.shape[0])
print('test size', labels_df.shape[0] - train_val_df.shape[0])

# 이미지 전처리
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 사전학습된 모델 불러오기
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))  # 마지막 FC 레이어 제거
model.eval()

def extract_features(img_path):
    img = Image.open(img_path).convert('RGB')
    img = preprocess(img)
    img = img.unsqueeze(0)
    with torch.no_grad():
        features = model(img)
    return features.numpy().flatten()

def process_paths(paths, desc):
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(extract_features, path): path for path in paths}
        results = []
        for future in tqdm(as_completed(futures), total=len(futures), desc=desc):
            results.append(future.result())
        return results

if not os.path.exists("train_val_features.pkl") or not os.path.exists("test_features.pkl"):
    # 멀티프로세싱을 사용하여 이미지에서 특징 추출
    train_val_paths = train_val_df['Paths'].tolist()
    test_paths = test_df['Paths'].tolist()

    train_val_features = process_paths(train_val_paths, desc="Extracting features train_val_df")
    test_features = process_paths(test_paths, desc="Extracting features test_df")

    # train_val 데이터 저장
    with open('train_val_features.pkl', 'wb') as f:
        pickle.dump({'paths': train_val_paths, 'features': train_val_features}, f)

    # test 데이터 저장
    with open('test_features.pkl', 'wb') as f:
        pickle.dump({'paths': test_paths, 'features': test_features}, f)
        
else:
    # 피클 파일에서 데이터 로드
    with open('train_val_features.pkl', 'rb') as f:
        train_val_data = pickle.load(f)

    with open('test_features.pkl', 'rb') as f:
        test_data = pickle.load(f)

  0%|          | 0/15 [00:00<?, ?it/s]

train_val size 86523
test size 25597


In [4]:
# # 특징과 레이블 분리
X_train = np.stack(train_val_data['features'])
y_train = train_val_df[disease_labels].values

X_test = np.stack(test_data['features'])
y_test = test_df[disease_labels].values

num_classes = 15

# XGBoost 모델 설정
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='mlogloss',
    num_class=num_classes,
    use_label_encoder=False,
    tree_method='gpu_hist',  # GPU를 사용하도록 설정
    gpu_id=3  # 특정 GPU를 지정
)

# tqdm 콜백 클래스 정의
class TqdmCallback(xgb.callback.TrainingCallback):
    def __init__(self, total, desc):
        self.total = total
        self.pbar = tqdm(total=total, desc=desc)

    def after_iteration(self, model, epoch, evals_log):
        self.pbar.update(1)
        if epoch == self.total - 1:
            self.pbar.close()
        return False

# OneVsRestClassifier로 멀티레이블 분류기 설정
epochs = 20
multilabel_model = OneVsRestClassifier(xgb_model)

columns = [	"Time",
			"Accuracy",
			"F1_macro",
			"Roc_Auc_macro"]

result_save_path = 'save'

init_time = datetime.now()
init_time = init_time.strftime('%m%d_%H%M')

init_df = pd.DataFrame(columns=columns)
csv_name = f'{result_save_path}/{init_time}_xgb_output.csv'
init_df.to_csv(csv_name, index=False)

In [5]:
# 모델 학습을 위한 맞춤 fit 함수 정의
def custom_fit(model, X_train, y_train, epochs):
    print("Model Fit Start")

    for i in range(y_train.shape[1]):
        print(f"Training for label {i}")
        model.estimator.fit(
            X_train, 
            y_train[:, i], 
            eval_set=[(X_train, y_train[:, i]), (X_test, y_test[:, i])],
            verbose=False, 
            callbacks=[TqdmCallback(epochs, desc=f'Training label {i}')]
        )

# 모델 학습
custom_fit(multilabel_model, X_train, y_train, epochs)

Model Fit Start
Training for label 0


Training label 0:   0%|          | 0/20 [00:00<?, ?it/s]

Training for label 1


Training label 1:   0%|          | 0/20 [00:00<?, ?it/s]

Training for label 2


...

Training label 12:   0%|          | 0/20 [00:00<?, ?it/s]

Training for label 13


Training label 13:   0%|          | 0/20 [00:00<?, ?it/s]

Training for label 14


Training label 14:   0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
# 예측
y_pred = multilabel_model.estimator.predict(X_test)
y_pred_proba = multilabel_model.estimator.predict_proba(X_test)  # AUC-ROC 계산을 위해 필요

# 정확도 및 F1 스코어 계산
accuracy = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')

# AUC-ROC 스코어 계산
roc_auc_micro = roc_auc_score(y_test, y_pred_proba, average='micro')
roc_auc_macro = roc_auc_score(y_test, y_pred_proba, average='macro')

print(f"Test Accuracy: {accuracy}")
# print(f"Test F1 Score (Micro): {f1_micro}")   # 라벨에 상관없이 전체적인 성능 평가 (==accuracy와 동일)
print(f"Test F1 Score (Macro): {f1_macro}")     # 모든 라벨이 유사한 중요도를 가져 단순 라벨들의 f1_score의 산술평균
# print(f"Test ROC AUC Score (Micro): {roc_auc_micro}") 
print(f"Test ROC AUC Score (Macro): {roc_auc_macro}")

# 각 레이블별 평가 결과 출력
for idx in range(y_test.shape[1]):
    accuracy_label = accuracy_score(y_test[:, idx], y_pred[:, idx])
    f1_label = f1_score(y_test[:, idx], y_pred[:, idx])
    roc_auc_label = roc_auc_score(y_test[:, idx], y_pred_proba[:, idx])
    print(f"Class {idx} - Accuracy: {accuracy_label}, F1 Score: {f1_label}, ROC AUC Score: {roc_auc_label}")


Test Accuracy: 0.022152764211760108
Test F1 Score (Macro): 0.020737174922037267
Test ROC AUC Score (Macro): 0.4948972736133399
Class 0 - Accuracy: 0.5162727095135768, F1 Score: 0.19546429267658716, ROC AUC Score: 0.48662868222177436
Class 1 - Accuracy: 0.47149833951943737, F1 Score: 0.11559333115397188, ROC AUC Score: 0.47049563607633793
Class 2 - Accuracy: 0.7612033600312561, F1 Score: 0.0, ROC AUC Score: 0.4988517266563171
Class 3 - Accuracy: 0.895878101191639, F1 Score: 0.0, ROC AUC Score: 0.4933712686216064
Class 4 - Accuracy: 0.9638601289314319, F1 Score: 0.0, ROC AUC Score: 0.4891093460708378
Class 5 - Accuracy: 0.9572963469427622, F1 Score: 0.0, ROC AUC Score: 0.49860483409573597
Class 6 - Accuracy: 0.9830044930650518, F1 Score: 0.0, ROC AUC Score: 0.5219617893755825
Class 7 - Accuracy: 0.8180113303379566, F1 Score: 0.0, ROC AUC Score: 0.49914744540312955
Class 8 - Accuracy: 0.9783160773588592, F1 Score: 0.0, ROC AUC Score: 0.4959062257145324
Class 9 - Accuracy: 0.95534284039851

In [7]:
#######################
now = datetime.now() 
csv_record_time = now.strftime('%Y%m%d_%H%M%S')
csv_accuracy = f"{accuracy:.4f}"
csv_f1_macro = f"{f1_macro:.4f}"
csv_roc_auc_macro = f"{roc_auc_macro:.4f}"

csv_data = [csv_record_time,
            csv_accuracy,
            csv_f1_macro,
            csv_roc_auc_macro]

df = pd.DataFrame([csv_data], columns=columns)
df.to_csv(csv_name, mode='a', header=False, index=False)
##########################